<a href="https://colab.research.google.com/github/RoheeAxel/3D-Engine/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

ds = load_dataset("dair-ai/emotion", "split")

# ds = load_dataset("google-research-datasets/go_emotions", "simplified")

train_set = ds["train"]
val_set = ds["validation"]
test_set = ds["test"]

train_set = train_set.map(lambda x: {"text": x["text"], "labels": [x["label"]]})
val_set = val_set.map(lambda x: {"text": x["text"], "labels": [x["label"]]})
test_set = test_set.map(lambda x: {"text": x["text"], "labels": [x["label"]]})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
import torch.nn as nn
import torch as torch

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

In [8]:
import torch.nn as nn

class Decepticon(nn.Transformer):
  def __init__(self, d_model: int, nb_head: int = 8, num_layer: int = 6):
    self.nhead: int = nb_head
    self.num_encoder_layers: int = num_layer
    self.num_decoder_layers: int = num_layer

    super(Decepticon, self).__init__(
        d_model=d_model,
        nhead=nb_head,
        num_encoder_layers=num_layer,
        num_decoder_layers=num_layer
    )

    self.pos_encoder = PositionalEncoding(d_model)

  def forward(self, src, tgt):
    src = self.pos_encoder(src)
    tgt = self.pos_encoder(tgt)

    return super(Decepticon, self).forward(src, tgt)

In [21]:
from torch.utils.data import DataLoader, Dataset

class DataSetLoader(Dataset):
  def __init__(self, src_data, tgt_data):
    self.src_data = src_data
    self.tgt_data = tgt_data

  def __len__(self):
    return len(self.src_data)

  def __getitem__(self, idx):
    return self.src_data[idx], self.tgt_data[idx]

train_dataset = DataSetLoader(train_set["text"], train_set["labels"])
val_dataset = DataSetLoader(val_set["text"], val_set["labels"])
test_dataset = DataSetLoader(test_set["text"], test_set["labels"])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

print(next(iter(train_loader)))

[('i always forgive and am still feeling hurt', 'im trying to go on how i feel hopefully next time i brave the scales i will have been good for a few days and will see a nicer number', 'ive been feeling so bothered lately', 'i am feeling horny so i ask her that lets go home', 'i can sink into the stillness to feel the gentle hum of that light there is pleasure in contrast', 'ill crawl into the kitchen feeling miserable and cook a fresh healthy meal', 'i felt happy when i received the letter telling me that i had been admitted to the university', 'i also always feel a little scared', 'i am very fascinated by it and don t feel so uptight by the many challenges life has because of it', 'i get out if bed and look in the mirror i feel brave', 'i feel about the plight of these dogs so its lovely to find a turkish vet who really cares', 'i can t get past feeling like a poseur to become an advocate i was ecstatic to see that keiko zoll has done it', 'i know how you all feel my mil has hated me

In [22]:
def tokenize(text):
    return text.lower().split()

def build_vocab(texts, max_vocab_size):
    from collections import Counter
    counter = Counter(token for text in texts for token in tokenize(text))
    most_common = counter.most_common(max_vocab_size - 2)
    vocab = {word: idx + 2 for idx, (word, _) in enumerate(most_common)}  # Reserve 0 and 1
    vocab["<PAD>"] = 0
    vocab["<UNK>"] = 1
    return vocab

def encode(text, vocab, max_len):
    tokens = tokenize(text)
    ids = [vocab.get(token, vocab["<UNK>"]) for token in tokens]
    if len(ids) > max_len:
        return ids[:max_len]
    return ids + [vocab["<PAD>"]] * (max_len - len(ids))

VOCAB_SIZE = 10000
vocab = build_vocab(train_set["text"], VOCAB_SIZE)

In [ ]:
import torch.optim as optim
import torch.nn as nn

d_model = 512
nb_head = 8
num_layer = 6

# Create model and wrap the transformer
model = Decepticon(d_model=d_model, nb_head=nb_head, num_layer=num_layer)

# Add an embedding layer and a final output projection (for classification or token prediction)
model.src_embed = nn.Embedding(VOCAB_SIZE, d_model)
model.tgt_embed = nn.Embedding(VOCAB_SIZE, d_model)
model.generator = nn.Linear(d_model, VOCAB_SIZE)  # Final layer for vocabulary logits

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)